In [221]:
import pandas as pd

# Suicide Rates Dataset

<a target="_blank" href="http://apps.who.int/gho/data/node.main.MHSUICIDEASDR?lang=en">Source</a>

Age-standardized suicide rates (per 100 000 population)
Suicide rates, age-standardized, per 100 000 population, by sex and year.
We considered only suicides amongst females for the year of 2014 (most recent data available)

In [393]:
suicides = pd.read_csv('suicide_core.csv').iloc[1:,0:3]

In [394]:
suicides.columns = ['country','sex','suicide_rate']

In [395]:
female_suicides = suicides[suicides.sex == 'Female'][['country','suicide_rate']]

In [396]:
female_suicides = female_suicides.assign(country = female_suicides.country.apply(lambda x: x.strip()))

In [397]:
len(female_suicides)

183

In [398]:
female_suicides.head()

,country,suicide_rate
3,Afghanistan,3.1
6,Albania,2.4
9,Algeria,1.3
12,Angola,14.3
15,Antigua and Barbuda,0.0


Check if there are null values: 

In [399]:
female_suicides.suicide_rate.isnull().sum()

0

# Drinks

<a target="_blank" href="https://www.kaggle.com/rjribeiro/alcohol-consumption">Source</a>

Pure alcohol consumption among persons (age 15+) in liters per capita per year

In [234]:
drinks = pd.read_csv('drinks.csv')

In [235]:
drinks_total = drinks[['country','total_litres_of_pure_alcohol']]

In [236]:
drinks_total = drinks_total.assign(country = drinks_total.country.apply(lambda x: x.strip()))

In [237]:
len(drinks_total)

193

In [238]:
drinks_total.head()

,country,total_litres_of_pure_alcohol
0,Afghanistan,0.0
1,Albania,4.9
2,Algeria,0.7
3,Andorra,12.4
4,Angola,5.9


Check if there are null values

In [239]:
drinks.total_litres_of_pure_alcohol.isnull().sum()

0

# GDI 

<a href="http://hdr.undp.org/en/composite/GDI#a" target="_blank">Source</a>

Countries are divided into five groups by absolute deviation from gender parity in HDI values. Data refer to 2015 or the most recent year if available. In calculating the HDI value, expected years of schooling is capped at 18 years.

We're using only GDI, HDI, schooling years and GNI variables. 

<b>Schooling years</b> Expected years a female devotes to studying

<b>GDI Value(2015)</b> Ratio of female to male HDI values

<b>HDI Female(2015)</b> A composite index measuring average achievement in three basic dimensions of human development—a long and healthy life, knowledge and a decent standard of living
Expected schooling years Female(2015) Number of years of schooling that a child of school entrance age can expect to receive if prevailing patterns of age-specific enrolment rates persist throughout the child’s life.

<b>Estimated GNI per capita - Female(2015)</b> Derived from the ratio of female to male wages, female and male shares of economically active population and GNI (in 2011 purchasing power parity terms).

In [251]:
gdi = pd.read_excel('GDI.xlsx')

In [252]:
gdi.columns

Index(['HDI rank', 'Country', 'GDI Value(2015)', 'GDI Group(2015)',
       'HDI Female(2015)', 'Expected schooling years Female(2015)',
       'Estimated GNI per capita -Female(2015)'],
      dtype='object')

In [253]:
gdi_selection = gdi[['Country', 'GDI Value(2015)', 'HDI Female(2015)', 'Expected schooling years Female(2015)',
    'Estimated GNI per capita -Female(2015)']]

In [254]:
gdi_selection.columns = ['country','gdi', 'hdi', 'schooling_years','gni']

In [255]:
gdi_selection = gdi_selection.assign(country = gdi_selection.country.apply(lambda x: x.strip()))

In [256]:
gdi_selection.head()

,country,gdi,hdi,schooling_years,gni
0,South Sudan,0.874,0.370,3.8,1286
1,Eritrea,0.874,0.370,4.4,1286
2,Niger,0.732,0.291,4.7,481
3,Central African Republic,0.776,0.306,5.8,482
4,Chad,0.765,0.340,5.8,1581


Check if there are missing values

In [257]:
gdi_selection.isnull().sum()

country            0
gdi                0
hdi                0
schooling_years    0
gni                0
dtype: int64

# Poverty

<a target="_blank" href="http://hdr.undp.org/en/composite/MPI">Source</a>

For poverty data sets we took the most recent years and imputed the data for 2014 from the years 2011-2013. For those that don't have any values for any years, we took the column average. 
TODO: add rationale (i.e. the remaining countires intuitively are good fit for the mean)

In [267]:
poverty = pd.read_excel('poverty.xlsx')
poverty_selection = poverty[['Country', 2014]]
poverty_selection.columns = ['country', 'poverty_index']
poverty_selection = poverty_selection.assign(country = poverty_selection.country.apply(lambda x: x.strip()))
poor_countries = poverty_selection['country']
means = poverty_selection["poverty_index"].fillna(poverty_selection["poverty_index"].mean())
poverty_selection = poverty_selection.assign(poverty_index = means)
poverty_selection.head()

,country,poverty_index
0,Afghanistan,0.293000
1,Albania,0.156609
2,Argentina,0.156609
3,Armenia,0.156609
4,Azerbaijan,0.156609


In [409]:
poverty = pd.read_csv('poverty2.csv')

# Happiness

<a href="https://ourworldindata.org/happiness-and-life-satisfaction" target="_blank">Source</a>

Life satisfaction is measured on a scale from 0 to 10 (0 - worst, 10 - best). 

In [412]:
happiness = pd.read_csv('happiness.csv')
happiness = happiness.assign(country = happiness.country.apply(lambda x: x.strip()))
happiness.columns

Index(['country', 'satisfaction_index'], dtype='object')

In [413]:
happiness[happiness.satisfaction_index.isnull()].head()

,country,satisfaction_index
2,Algeria,NaN
3,Angola,NaN
4,Antigua and Barbuda,NaN
5,Arab World,NaN
11,Bahamas,NaN


Rationale for missing values. Values are missing only for the poor, undeveloped countires. Examples of them are Algeria, Angola. In the previous dataset on poverty we had only poor and hence unhappy countries (TODO: add link to research that shows that richer -> happier in general). So we're going to select all the countries that are considered poor and take their average for imputation.

In [414]:
poor_countries_mean = pd.merge(pd.DataFrame(poor_countries), happiness, how='left').satisfaction_index.mean()

In [415]:
happiness.fillna(poor_countries_mean, inplace = True)

In [416]:
happiness.head()

,country,satisfaction_index
0,Afghanistan,3.982855
1,Albania,4.606651
2,Algeria,4.778841
3,Angola,4.778841
4,Antigua and Barbuda,4.778841


# Unemployement rates

<a href="https://www.kaggle.com/uddipta/world-bank-unemployment-data-19912017" target="_blank">Source</a>

This data contains unemployment rates for different countries, across different regions with different income groups, supplied by the world bank. 

In [426]:
unempl = pd.read_excel('unemployement_rates.xlsx')

In [427]:
unempl_selection = unempl[['Country Name', 2015]]
unempl_selection.columns = ['country', 'unemployment_rate']

In [428]:
unempl_selection.unemployment_rate.isnull().sum()

0

In [429]:
unempl_selection.head()

,country,unemployment_rate
0,Afghanistan,8.6
1,Albania,17.1
2,Algeria,11.2
3,Angola,6.2
4,Argentina,7.2


In [430]:
unempl_selection.unemployment_rate.sum()

649888.061718299

# Health services 

<a target="_blank" href="http://apps.who.int/gho/data/node.main.REPWOMEN39">Source</a>

Married or in-union women of reproductive age who have their need for family planning satisfied with modern methods (%)

In [433]:
health = pd.read_csv("health_services.csv")

In [434]:
health_selection = health.iloc[:,[0,2]]
health_selection.columns = ['country', 'access_to_family_planning']

In [435]:
health_selection.head()

,country,access_to_family_planning
0,Afghanistan,47.0
1,Albania,12.9
2,Algeria,77.2
3,Angola,24.2
4,Antigua and Barbuda,58.2


# GII

<a target="_blank" href="http://hdr.undp.org/en/composite/GII">Source</a>

Data are average of period estimates for 2010-2015. In calculating the Gender Inequality Index, a value of 0.1 percent was used.

<b> GII Value</b>  
A composite measure reflecting inequality in achievement between women and men in three dimensions: reproductive health, empowerment and the labour market.

<b>Maternal mortality ratio (deaths per 100,000 live births)</b>  
Number of deaths due to pregnancy-related causes per 100,000 live births.

<b>Adolescent birth rate (birth per 1,000 women ages 15-19)</b>  
 Number of births to women ages 15–19 per 1,000 women ages 15–19.

<b>Labour force participation rate(% ages 15 and older) </b>  
Proportion of the working-age population (ages 15 and older) that engages in the labour market, either by working or actively looking for work, expressed as a percentage of the working-age population.


In [497]:
gii = pd.read_excel('GII.xlsx')

In [498]:
gii_selection = gii.iloc[:,[0,1,3,4,5]]

In [499]:
gii_selection.columns = ['country','gii','maternal_mortality','adolescent_birth','labor_participation']

In [500]:
gii_selection.head()

,country,gii,maternal_mortality,adolescent_birth,labor_participation
0,Afghanistan,0.667,396,74.0,19.1
1,Albania,0.267,29,21.7,40.3
2,Algeria,0.429,140,10.6,16.8
3,Andorra,0.078,4,7.1,54.7
4,Angola,0.672,477,164.3,59.9


# Human Development Index

In [452]:
hdi = pd.read_csv('human_development_index.csv')

In [453]:
hdi_selection = hdi[['Country', '2015']]

In [454]:
hdi_selection.columns = ['country', 'human_development']

In [455]:
hdi_selection.head()

,country,human_development
0,Afghanistan,0.479
1,Albania,0.764
2,Algeria,0.745
3,Andorra,0.858
4,Angola,0.533


# Mental Health

<a href="http://apps.who.int/gho/data/node.main.MHFAC?lang=en" target="_blank">Source</a>

In [463]:
mental = pd.read_csv('mental_health.csv')

In [464]:
mental.columns = ['country', 'num_mental_hospitals']

In [465]:
mental.head()

,country,num_mental_hospitals
0,Afghanistan,0.00
1,Albania,0.06
2,Algeria,0.04
3,Andorra,0.00
4,Angola,0.01


# Sanitation

<a target="_blank" href="http://www.who.int/gho/mdg/environmental_sustainability/sanitation/en/">Source</a>

In [469]:
sanitation = pd.read_csv('sanitation.csv')

In [470]:
sanitation.columns = ['country', 'basic_sanitation_percentage']

In [471]:
sanitation.head()

,country,basic_sanitation_percentage
0,Afghanistan,39.0
1,Albania,98.0
2,Algeria,87.0
3,Angola,39.0
4,Antigua and Barbuda,88.0


# Life course

<a target="_blank" href="http://hdr.undp.org/en/composite/Dashboard1">Source</a>

<b>** GER = Gross enrolment ratio</b>
All the ratios and percentages are for the year 2015

<b>GER-Pre-primary-female(2015) - column name (preprimary)</b>
Total enrolment of girls in a given level of education pre-primary regardless of age, expressed as a percentage of the official school-age female population for the same level of education.

<b>GER-Primary-female(2015) - column name (primary)</b>
Total enrolment of girls in a given level of education primary regardless of age, expressed as a percentage of the official school-age female population for the same level of education.

<b>GER-Secondary-female(2015) - column name (secondary)</b>
Total enrolment of girls in a given level of education secondary, regardless of age, expressed as a percentage of the official school-age female population for the same level of education.	

<b>Youth unemployment rate - ratio (2015)	- column name (youth_unempl)</b>
Ratio of the percentage of the female labour force population ages 15–24 that is not in paid employment or self-employed but is available for work and is actively seeking paid employment or self-employment to the percentage of the male labour force population ages 15-24 that is not in paid employment or self-employed but is available for work and is actively seeking paid employment or self-employment.

<b>Total unemployment rate - ratio(2015) - column name(total_unempl)</b>
Ratio of the percentage of the female labour force population ages 15 and older that is not in paid employment or self-employed but is available for work and is actively seeking paid employment or self-employment to the percentage of the male labour force population ages 15 and older that is not in paid employment or self-employed but is available for work and is actively seeking paid employment or self-employment.

<b>Life expectancy at age 50, female(2015) - column name (life_expectancy_50)</b>
Additional number of years that a 50-year-old woman could expect to live if prevailing patterns of female age-specific mortality rates stay the same throughout the rest of her life.


In [477]:
life_course = pd.read_excel('life_course.xlsx')

In [478]:
life_course_selection = life_course.iloc[:,1:]
life_course_selection.columns = ['country', 'preprimary', 'primary','secondary', 'youth_unempl', 'total_unempl', 'life_expectancy_50']

In [479]:
life_course_selection.head()

,country,preprimary,primary,secondary,youth_unempl,total_unempl,life_expectancy_50
0,Afghanistan,66,92,40.0,1.01,1.50,24.0
1,Albania,87,111,93.0,0.93,0.84,32.5
2,Algeria,79,115,102.0,1.15,1.95,31.1
3,Andorra,101,102,97.0,0.96,0.89,34.7
4,Angola,94,100,23.0,1.08,1.21,23.5


# Sunshine hours

<a href="http://data.un.org/Data.aspx?q=Korea&d=CLINO&f=ElementCode%3A15%3BCountryCode%3AKO" target="_blank">Source</a>

In [485]:
sunshine = pd.read_excel("sunshine.xlsx")

In [489]:
sunshine = sunshine[['country', 'sunshine_hours']]

In [490]:
sunshine.head()

,country,sunshine_hours
0,Afghanistan,3085.0
1,Albania,2456.8
2,Algeria,2847.0
3,Angola,2341.0
4,Antigua and Barbuda,2760.0


# Datasets Merging

In [377]:
from functools import reduce

In [378]:
datasets = [female_suicides, drinks_total,
            gdi_selection, poverty_selection,
            happiness, unempl_selection, 
            health_selection, gii_selection,
            hdi_selection, mental,
            sanitation, life_course_selection,
            sunshine]

In [379]:
df_final = reduce(lambda left,right: pd.merge(left,right,on='country', how='left'), datasets)

In [400]:
# merge suicides and alcohol
s_dr = pd.merge(female_suicides, drinks_total, how='left')
s_dr.total_litres_of_pure_alcohol.isnull().sum()

5

In [401]:
s_dr[s_dr.total_litres_of_pure_alcohol.isnull()]

,country,suicide_rate,total_litres_of_pure_alcohol
44,Democratic People's Republic of Korea,14.3,NaN
45,Democratic Republic of Congo,7.3,NaN
55,Eswatini,9.6,NaN
131,Korea,13.4,NaN
151,South Sudan,5.4,NaN


In [450]:
add_gdi = pd.merge(s_dr, gdi_selection, how='left')
#add_gdi[add_gdi.isnull().any(axis=1)]

In [449]:
add_poverty = pd.merge(add_gdi, poverty, how = 'left')
#add_poverty[add_poverty.isnull().any(axis=1)]

In [448]:
add_happiness = pd.merge(add_poverty, happiness, how = 'left')
#add_happiness[add_happiness.isnull().any(axis=1)]

In [447]:
add_unemployment = pd.merge(add_happiness, unempl_selection, how='left')
#add_unemployment[add_unemployment.isnull().any(axis=1)]

In [446]:
add_health = pd.merge(add_unemployment, health_selection, how='left')
#add_health[add_health.isnull().any(axis=1)]

In [501]:
add_gii = pd.merge(add_health, gii_selection, how='left')
#add_gii[add_gii.isnull().any(axis=1)]

In [502]:
add_hdi = pd.merge(add_gii, hdi_selection, how='left')
#add_hdi[add_gii.isnull().any(axis=1)]

In [503]:
add_mental = pd.merge(add_hdi, mental, how='left')
#add_mental[add_mental.isnull().any(axis=1)]

In [504]:
add_sanitation = pd.merge(add_mental, sanitation, how='left')
#add_sanitation[add_sanitation.isnull().any(axis=1)]

In [505]:
add_lifecourse = pd.merge(add_sanitation, life_course_selection, how='left')
#add_lifecourse[add_lifecourse.isnull().any(axis=1)]

In [507]:
add_sunshine = pd.merge(add_lifecourse, sunshine, how='left')
full_dataset = add_sunshine[add_sunshine.notnull().all(axis=1)]

In [508]:
full_dataset.to_csv('full_suicide_dataset.csv', index=False)